In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

from scipy.stats import norm
from scipy.stats import multivariate_normal

In [3]:
#load data into memory
data = pd.DataFrame(data = np.genfromtxt('sim3.dat', delimiter='  '), columns=['i','t','y_t','p_t'])

#set up lag
shift_data = data[['i','y_t','p_t']]
shift_data['t'] = data['t'] + 1
data = data.merge(shift_data,how='left',on=['i','t'],suffixes=['','-1'])
data = data.dropna()

# Part d

In [93]:
#fix alpha ahead of time

#global variables
NO_SIMS = 20
ALPHA = np.random.normal(0,1 ,(1900,NO_SIMS))
    

class part_d(GenericLikelihoodModel):
    """class for evaluating question 3 part d"""
    
    def nloglikeobs(self, params, alpha=ALPHA, v=False):
        t0, t1, t2, sigma = params
        y = self.endog
        p,y_lag = self.exog.transpose()
    
        #calculate the mean 'delta' for the inside good
        U1 = t0 + t1*p + t2*y_lag
        U1 =  np.tile(U1 ,(NO_SIMS,1)).transpose()
        U1 = np.exp(U1+ sigma*alpha)
        
        #calculate ll, for each simulation
        
        y = np.tile(y ,(NO_SIMS,1)).transpose() 
        likelihood_sims = np.log( y*U1/(1+U1) + (1-y)/(1+U1))
       

        likelihood = likelihood_sims.sum(axis=0)        
        if v: raise Exception('Stop drop and roll')
        return - likelihood.sum()

    
    def fit(self, start_params=None, maxiter=1000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        if start_params == None:
            start_params = [-.798,.5,.5,.5]
        return super(part_d, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)
    

model_d = part_d(data['y_t'],data[['p_t','y_t-1']])

result_d = model_d.fit()
print(result_d.summary(xname=['theta_0', 'theta_1', 'theta_2', 'sigma']))

Optimization terminated successfully.
         Current function value: 12.877600
         Iterations: 140
         Function evaluations: 235
                                part_d Results                                
Dep. Variable:                    y_t   Log-Likelihood:                -24467.
Model:                         part_d   AIC:                         4.894e+04
Method:            Maximum Likelihood   BIC:                         4.895e+04
Date:                Thu, 25 Oct 2018                                         
Time:                        16:51:11                                         
No. Observations:                1900                                         
Df Residuals:                    1898                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------

# Part e

# Part f

In [88]:
class part_f(GenericLikelihoodModel):
    """class for evaluating question 3 part f"""
    
    def nloglikeobs(self, params, v=False):
        t0, t1, t2 = params
        y = self.endog
        p,y_lag = self.exog.transpose()
    
        #calculate the mean 'delta' for the inside good
        U1 = t0 + t1*p + t2*y_lag
        U1 = np.exp(U1)
        
        #calculate ll, for each simulation
        likelihood_sims = np.log(y*U1/(1+U1) + (1-y)/(1+U1))
        likelihood = likelihood_sims.sum(axis=0)
        
        if v: raise Exception('Stop drop and roll')
        return - likelihood.sum()

    
    def fit(self, start_params=None, maxiter=1000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        if start_params == None:
            start_params = [-.948,.5,.5]
        return super(part_f, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)
    

model_f = part_f(data['y_t'],data[['p_t','y_t-1']])

result_f = model_f.fit()
print(result_f.summary(xname=['theta_0', 'theta_1', 'theta_2']))

Optimization terminated successfully.
         Current function value: 0.643884
         Iterations: 84
         Function evaluations: 148
                                part_f Results                                
Dep. Variable:                    y_t   Log-Likelihood:                -1223.4
Model:                         part_f   AIC:                             2451.
Method:            Maximum Likelihood   BIC:                             2462.
Date:                Thu, 25 Oct 2018                                         
Time:                        16:31:57                                         
No. Observations:                1900                                         
Df Residuals:                    1898                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------

# Part g